In [1]:
!pip install transformers

In [2]:
import torch
import re
from transformers import AutoTokenizer, ViTFeatureExtractor, VisionEncoderDecoderModel


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
encoder_checkpoint = "nlpconnect/vit-gpt2-image-captioning"
decoder_checkpoint = "nlpconnect/vit-gpt2-image-captioning"
model_checkpoint = "nlpconnect/vit-gpt2-image-captioning"
feature_extractor = ViTFeatureExtractor.from_pretrained(encoder_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(decoder_checkpoint)
model = VisionEncoderDecoderModel.from_pretrained(model_checkpoint).to(device)


Using cuda device


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

In [4]:
def predict(image,max_length=64, num_beams=4):
  image = feature_extractor(image, return_tensors="pt").pixel_values.to(device)
  clean_text = lambda x: x.replace('<|endoftext|>','').split('\n')[0]
  caption_ids = model.generate(image, max_length = max_length)[0]
  caption_text = clean_text(tokenizer.decode(caption_ids))
  return caption_text

In [5]:
from PIL import Image

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pandas as pd
import numpy as np
train = pd.read_json("/content/drive/MyDrive/meme/data/train.jsonl",lines=True)
test = pd.read_json("/content/drive/MyDrive/meme/data/test.jsonl",lines=True)
val = pd.read_json("/content/drive/MyDrive/meme/data/dev.jsonl",lines=True)

In [13]:
base_path='/content/drive/MyDrive/meme/data/'

In [14]:
train=val

In [15]:
train.head()

,id,img,label,text
0,8291,img/08291.png,1,white people is this a shooting range
1,46971,img/46971.png,1,bravery at its finest
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...
4,80243,img/80243.png,1,mississippi wind chime


In [16]:
train.shape

(500, 4)

In [17]:
capt_Text=[]
for ind in train.index:
  im = Image.open(f"{base_path}{train['img'][ind]}")
  im = im.convert('RGB')
  capt_text=predict(im)
  if ind%100==0:
    print(f"Caption text for  index {ind} is {capt_text}")
  capt_Text.append(capt_text)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Caption text for  index 0 is a woman sitting on a curb eating a piece of food 
Caption text for  index 100 is a woman and a man are in a kitchen 
Caption text for  index 200 is a collage of photos showing a collage of a television set 
Caption text for  index 300 is a person holding a hand up to a picture of a person 
Caption text for  index 400 is a woman sitting in a chair with a laptop 


In [18]:
train['capt_text']=capt_Text

In [19]:
train.head()

,id,img,label,text,capt_text
0,8291,img/08291.png,1,white people is this a shooting range,a woman sitting on a curb eating a piece of food
1,46971,img/46971.png,1,bravery at its finest,a man holding a sign with a picture of a woman
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,a woman is sitting at a desk with a laptop
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,a man wearing a costume holding a large piece ...
4,80243,img/80243.png,1,mississippi wind chime,a crowd of people standing around a group of p...


In [20]:
train.to_csv(f'{base_path}val_caption.csv')

In [21]:
train.to_json(f'{base_path}val_with_caption.json',orient='records')

In [22]:
df=pd.read_json(f'{base_path}val_with_caption.json')

In [23]:
df.head()

,id,img,label,text,capt_text
0,8291,img/08291.png,1,white people is this a shooting range,a woman sitting on a curb eating a piece of food
1,46971,img/46971.png,1,bravery at its finest,a man holding a sign with a picture of a woman
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,a woman is sitting at a desk with a laptop
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,a man wearing a costume holding a large piece ...
4,80243,img/80243.png,1,mississippi wind chime,a crowd of people standing around a group of p...


In [25]:
df.shape

(500, 5)